In [1]:
import os
import sys
from glob import glob
import requests

import time
import datetime

import pandas as pd
import numpy as np
import json

import threading

import ta
import redis
import schedule
import shutil

from cryptofeed.defines import TRADES, L2_BOOK, BID, ASK

from algos.daddy.live_trader import liveTrading
from algos.daddy.trade_analysis import get_trades, process_data, get_details
from algos.daddy.historic import single_price_from_rest
from algos.daddy.plot import create_chart
from algos.daddy.trades import update_trades, run_backtest
from algos.daddy.bot import daddyBot

from utils import print

r = redis.Redis(host='localhost', port=6379, db=0)



/home/warproxxx/.local/lib/python3.8/site-packages/arctic/_util.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/home/warproxxx/.local/lib/python3.8/site-packages/arctic/store/_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [2]:
symbol='XBT'
TESTNET=False
config_file='algos/daddy/exchanges.csv'
parameter_file="algos/daddy/parameters.json"

In [3]:
bot = daddyBot(symbol=symbol, config_file=config_file, parameter_file=parameter_file, TESTNET=TESTNET)

In [4]:
lts = bot.lts

In [5]:
for idx, details in bot.EXCHANGES.iterrows():
    if details['trade'] == 1:
        break

In [6]:
details

name                      bitmex_1
exchange                    bitmex
types                         book
name_type                     fund
cryptofeed_name             Bitmex
ccxt_symbol                BTC/USD
symbol                      XBTUSD
cryptofeed_symbol           XBTUSD
subaccount                     NaN
trade                            1
collateral                     BTC
max_trade                      0.5
buy_method           10sec_average
sell_method          10sec_average
html_color                  danger
Name: 1, dtype: object

In [7]:
lt = lts[details['name']]

In [8]:
lt

In [9]:
current_pos, avgEntryPrice, _ = lt.get_position()

In [10]:
current_pos

'LONG'

In [11]:
avgEntryPrice

56883.0

In [12]:
pd.concat([pd.read_csv('algos/daddy/exchanges.csv'), pd.read_csv('algos/eth_daddy/exchanges.csv')])

,name,exchange,types,name_type,cryptofeed_name,ccxt_symbol,symbol,cryptofeed_symbol,subaccount,trade,collateral,max_trade,buy_method,sell_method,html_color
0,bitmex,bitmex,book,fund,Bitmex,BTC/USD,XBTUSD,XBTUSD,NaN,0,BTC,0.5,10sec_average,10sec_average,danger
1,bitmex_1,bitmex,book,fund,Bitmex,BTC/USD,XBTUSD,XBTUSD,NaN,1,BTC,0.5,10sec_average,10sec_average,danger
2,bybit,bybit,book,fund,Bybit,BTC/USD,BTCUSD,BTC-USD,NaN,0,BTC,0.0,10sec_average,10sec_average,dark
3,ftx,ftx,book,fund,FTX,BTC-PERP,BTCUSDT,BTC-PERP,daddy_BTC,0,USD,0.0,10sec_average,10sec_average,info
4,okex,okex,book,fund,OKEx,BTC-USD-SWAP,BTCUSD,BTC-USD-SWAP,NaN,0,BTC,0.0,10sec_average,10sec_average,secondary
5,huobi_swap,huobi_swap,book,fund,HuobiSwap,BTC-USD,BTCUSD,BTC-USD,NaN,0,BTC,0.0,10sec_average,10sec_average,primary
6,binance_futures,binance_futures,book,fund,BinanceFutures,BTC/USDT,BTCUSDT,BTC-USDT,NaN,0,USDT,0.0,10sec_average,10sec_average,warning
0,eth_ftx,ftx,book,fund,FTX,ETH-PERP,ETHUSDT,ETH-PERP,daddy_ETH,0,USD,0.0,10sec_average,10sec_average,info
